# Report for ForestQuery into Global Deforestation, 1990 to 2016

ForestQuery is on a mission to combat deforestation around the world and to raise awareness about this topic and its impact on the environment. The data analysis team at ForestQuery has obtained data from the World Bank that includes forest area and total land area by country and year from 1990 to 2016, as well as a table of countries and the regions to which they belong.

The data analysis team has used SQL to bring these tables together and to query them in an effort to find areas of concern as well as areas that present an opportunity to learn from successes.


In [ ]:
print("Executing postgresql setup...")
%reload_ext sql
%sql postgresql://jadug.parusa:@localhost/deforestation
%sql SELECT * FROM forest_area WHERE forest_area_sqkm>0 ORDER BY 3 DESC, 4 DESC LIMIT 3

In [ ]:
%%sql

-- #DROP VIEW IF EXISTS deforestation_tab;
CREATE OR REPLACE VIEW deforestation_tab AS (
    SELECT
        f.country_code,
        f.country_name,
        f.year,
        f.forest_area_sqkm,
        (2.59 * l.total_area_sqmi) AS total_area_sqkm,
        r.region,
        r.income_group
    FROM forest_area AS f
    FULL OUTER JOIN
        land_area AS l
        ON
            f.country_code = l.country_code
        AND
            f.year = l.year
    FULL OUTER JOIN
        regions AS r
        ON
            f.country_code = r.country_code
);
SELECT *, COUNT(*) over()
FROM deforestation_tab
WHERE forest_area_sqkm>0
GROUP BY 1,2,3,4,5,6,7
ORDER BY 3 DESC, 4 DESC
LIMIT 3;

## 1. GLOBAL SITUATION

#### Question 1.1: How was the World's 1990 total forest area compared to 2016?

In [ ]:
%%sql
-- # filter country (world), year (1990 or 2016), calculate (area, difference, and percentage)
DROP VIEW IF EXISTS world_forest_loss;
CREATE VIEW world_forest_loss AS (
    WITH
    world_1990_2016 AS (
        SELECT
            country_name,
            year,
            forest_area_sqkm
        FROM
            deforestation_tab
        WHERE
            country_name = 'World'
            AND (
                year = 1990
                OR year = 2016
            )
    ),
    forest_1990 AS (
        SELECT
            forest_area_sqkm AS base_1990_sqkm
        FROM
            world_1990_2016
        WHERE
            year = 1990
    )

    SELECT
        t.country_name,
        t.year,
        t.forest_area_sqkm,
        base_1990_sqkm,
        (t.forest_area_sqkm - base_1990_sqkm) AS forest_change_sqkm,
        100*ROUND((t.forest_area_sqkm - base_1990_sqkm) / base_1990_sqkm, 3) AS forest_change_percent
    FROM
        world_1990_2016 AS t,
        forest_1990
    ORDER BY 2
);

SELECT * FROM world_forest_loss

#### Question 1.2: How comparable is this forest area loss against a single country size?

In [ ]:
%%sql
-- # filter year (2016), filter and order (by total_area_sqkm)
WITH base_2016_loss AS (
    SELECT
        forest_change_sqkm AS world_loss_sqkm
    FROM
        world_forest_loss
    WHERE
        year = 2016
)

SELECT
    country_name,
    year,
    region,
    total_area_sqkm
FROM
    deforestation_tab AS d,
    base_2016_loss
WHERE
    year = 2016
    AND d.total_area_sqkm < -(world_loss_sqkm)
ORDER BY 4 DESC
LIMIT 3

According to the World Bank, the total forest area of the world was `41,282,694.9 sqkm` in 1990. As of 2016, the most recent year for which data was available, that number had fallen to `39,958,245.9 sqkm`, a loss of `-1,324,449.0 sqkm`, or `-3.200%`.

The forest area lost over this time period is slightly more than the entire land area of `Peru` listed for the year 2016 (which is `1,279,999.9891 sqkm`).


## 2. REGIONAL OUTLOOK

#### Question 2.1:
#### How many percent of World's land area was designated as forest, in 2016 and 1990?
#### Which region had the highest & the lowest forestation percentage respectively in 2016 and 1990?

In [ ]:
%%sql
-- # filter year (1990 or 2016), calculate (forest area, land area, and percentage),
-- # select world's, highest region, lowest region
-- # apply union
DROP VIEW IF EXISTS region_forestation;
CREATE VIEW region_forestation AS (
    SELECT
        region,
        year,
        ROUND(100 * SUM(forest_area_sqkm) / SUM(total_area_sqkm), 3) AS forestation_percent,
        ROUND(SUM(forest_area_sqkm),3) AS region_forest_sqkm,
        ROUND(SUM(total_area_sqkm),3) AS region_land_sqkm
    FROM
        deforestation_tab
    WHERE
        year = 2016
        OR year = 1990
        AND COALESCE(forest_area_sqkm,0) > 0 -- excluding countries having empty forest_area_sqkm
        AND COALESCE(total_area_sqkm,0) > 0 -- excluding countries having empty total_area_sqkm
    GROUP BY 1,2
    ORDER BY 2, 3 DESC
);
WITH
world_forestation AS (
    SELECT * FROM region_forestation WHERE region = 'World' LIMIT 2
),
highest_2016 AS (
    SELECT * FROM region_forestation WHERE year = 2016 ORDER BY 3 DESC LIMIT 1
),
lowest_2016 AS (
    SELECT * FROM region_forestation WHERE year = 2016 ORDER BY 3 LIMIT 1
),
highest_1990 AS (
    SELECT * FROM region_forestation WHERE year = 1990 ORDER BY 3 DESC LIMIT 1
),
lowest_1990 AS (
    SELECT * FROM region_forestation WHERE year = 1990 ORDER BY 3 LIMIT 1
)

SELECT * FROM world_forestation
UNION
SELECT * FROM highest_2016
UNION
SELECT * FROM lowest_2016
UNION
SELECT * FROM highest_1990
UNION
SELECT * FROM lowest_1990

ORDER BY 1 DESC,2

In 2016, the percent of the total land area of the world designated as forest was `31.376%`. The region with the highest relative forestation was `Latin America & Caribbean`, with `46.162%`, and the region with the lowest relative forestation was `Middle East & North Africa`, with `2.068%` forestation.

In 1990, the percent of the total land area of the world designated as forest was `32.422%`. The region with the highest relative forestation was `Latin America & Caribbean`, with `51.031%`, and the region with the lowest relative forestation was `Middle East & North Africa`, with `1.777%` forestation.


#### Question 2.2:
#### From 1990 to 2016, which regions had increasing forestation & which regions had decreasing forestation?
#### How did those regional increase & decrease impact to the overall World' forestation?

In [ ]:
%%sql
-- # self join to combine percentage (1990 or 2016)
WITH
rf_selfjoin AS (
    SELECT
        rf_1990.region,
        rf_1990.forestation_percent AS forestation_1990_percent,
        rf_2016.forestation_percent AS forestation_2016_percent
    FROM region_forestation AS rf_1990
    INNER JOIN region_forestation AS rf_2016
    ON
        rf_1990.region = rf_2016.region
        AND rf_1990.year = 1990
        AND rf_2016.year = 2016
)

SELECT
    region,
    forestation_1990_percent,
    forestation_2016_percent,
    ROUND(100*(forestation_2016_percent - forestation_1990_percent)/forestation_1990_percent, 3) AS percent_change
FROM
    rf_selfjoin
ORDER BY 4

The only regions of the world that decreased in percent forest area from 1990 to 2016 were `Latin America & Caribbean` (dropped from `51.031%` to `46.162%`) and `Sub-Saharan Africa` (`32.190%` to `28.788%`).

All other regions actually increased in forest area over this time period. However, the drop in forest area in the two aforementioned regions was so large, the percent forest area of the world decreased over this time period from `32.422%` to `31.376%`. 


## 3. COUNTRY-LEVEL DETAIL

### A. SUCCESS STORIES

#### Question 3.1:
#### Which top 2 countries had the largest increase of forest area from 1990 to 2016?

In [ ]:
%%sql
-- # filter year (1990 or 2016), calculate (forest area, land area, and percentage),
-- # self join to combine percentage (1990 or 2016)
-- # select highest countries

DROP VIEW IF EXISTS country_forest_area;
CREATE VIEW country_forest_area AS (
    SELECT
        d_1990.country_name,
        d_1990.region,
        ROUND(SUM(d_1990.total_area_sqkm), 3) AS land_1990_sqkm,
        ROUND(SUM(d_2016.total_area_sqkm), 3) AS land_2016_sqkm,
        ROUND(SUM(d_1990.forest_area_sqkm), 3) AS forest_1990_sqkm,
        ROUND(SUM(d_2016.forest_area_sqkm), 3) AS forest_2016_sqkm,
        ROUND(100 * SUM(d_1990.forest_area_sqkm) / SUM(d_1990.total_area_sqkm), 3) AS forestation_1990_percent,
        ROUND(100 * SUM(d_2016.forest_area_sqkm) / SUM(d_2016.total_area_sqkm), 3) AS forestation_2016_percent
    FROM
        deforestation_tab AS d_1990
    INNER JOIN
        deforestation_tab AS d_2016
    ON
        d_1990.country_name = d_2016.country_name
        AND d_1990.year = 1990
        AND d_2016.year = 2016
    WHERE
        COALESCE(d_1990.total_area_sqkm, 0) > 0 -- excluding countries having empty land_area_sqkm
        AND COALESCE(d_2016.total_area_sqkm, 0) > 0 -- excluding countries having empty land_area_sqkm
        AND COALESCE(d_1990.forest_area_sqkm, 0) > 0 -- excluding countries having empty forest_area_sqkm
        AND COALESCE(d_2016.forest_area_sqkm, 0) > 0 -- excluding countries having empty forest_area_sqkm
    GROUP BY 1,2
);

SELECT
    country_name,
    region,
    land_2016_sqkm,
    forest_1990_sqkm,
    forest_2016_sqkm,
    (forest_2016_sqkm - forest_1990_sqkm) AS forest_sqkm_change
FROM
    country_forest_area
ORDER BY
    forest_sqkm_change DESC
LIMIT 5

There is one particularly bright spot in the data at the country level, `China`. This country actually increased in forest area from 1990 to 2016 by `527,229.062 sqkm`. It would be interesting to study what has changed in this country over this time to drive this figure in the data higher.

The country with the next largest increase in forest area from 1990 to 2016 was the `United States`, but it only saw an increase of `79,200.000 sqkm`, much lower than the figure for `China`.


#### Question 3.2:
#### Which top 2 countries had the largest land area in 2016?

In [ ]:
%%sql

SELECT
    country_name,
    region,
    land_2016_sqkm,
    (forest_2016_sqkm - forest_1990_sqkm) AS forest_sqkm_change,
    forestation_1990_percent,
    forestation_2016_percent,
    ROUND(100*(forestation_2016_percent - forestation_1990_percent)/forestation_1990_percent, 3) AS percent_change
FROM
    country_forest_area
WHERE
    country_name != 'World'
ORDER BY
    land_2016_sqkm DESC
LIMIT 5

#### Question 3.3:
#### Which top 5 countries had the largest increase of forestation percentage from 1990 to 2016?

In [ ]:
%%sql

SELECT
    country_name,
    region,
    land_2016_sqkm,
    (forest_2016_sqkm - forest_1990_sqkm) AS forest_sqkm_change,
    forestation_1990_percent,
    forestation_2016_percent,
    ROUND(100*(forestation_2016_percent - forestation_1990_percent)/forestation_1990_percent, 3) AS percent_change
FROM
    country_forest_area
ORDER BY
    percent_change DESC
LIMIT 5

`Russian Federation` (`16,376,870.000 sqkm`) and `China` (`9,388,210.011 sqkm`) are of course very large countries in total land area, so when we look at the largest percent change in forest area from 1990 to 2016, we aren’t surprised to find a much smaller country listed at the top.

`Iceland` (`100,249.990 sqkm`) increased in forest area by `213.043%` from 1990 to 2016. 


### B. LARGEST CONCERNS

#### Question 3.4: Which countries are seeing deforestation to the largest degree from 1990 to 2016? (by sqkm decrease in forest area)

The following 3 countries had the largest decrease in forest area over the time period under consideration:

In [ ]:
%%sql

SELECT
    country_name,
    region,
    land_2016_sqkm,
    forest_1990_sqkm,
    forest_2016_sqkm,
    (forest_2016_sqkm - forest_1990_sqkm) AS forest_sqkm_change
FROM
    country_forest_area
WHERE
    country_name != 'World'
ORDER BY
    forest_sqkm_change
LIMIT 5

#### Question 3.5: Which countries are seeing deforestation to the largest degree from 1990 to 2016? (by percent decrease)

In [ ]:
%%sql

SELECT
    country_name,
    region,
    land_2016_sqkm,
    (forest_2016_sqkm - forest_1990_sqkm) AS forest_sqkm_change,
    forestation_1990_percent,
    forestation_2016_percent,
    ROUND(100*(forestation_2016_percent - forestation_1990_percent)/forestation_1990_percent, 3) AS percent_change
FROM
    country_forest_area
ORDER BY
    percent_change
LIMIT 5

When we consider countries that decreased in forest area percentage the most between 1990 and 2016, we find that four of the top 5 countries on the list are in the region of `Sub-Saharan Africa`. The countries are `Togo`, `Nigeria`, `Uganda`, and `Mauritania`. The 5th country on the list is `Honduras`, which is in the `Latin America & Caribbean` region. 

From the above analysis, we see that `Nigeria` is the only country that ranks in the top 5 both in terms of absolute square kilometer decrease in forest as well as percent decrease in forest area from 1990 to 2016. Therefore, this country has a significant opportunity ahead to stop the decline and hopefully spearhead remedial efforts.


### C. QUARTILES

In [ ]:
%%sql

DROP VIEW IF EXISTS country_quartile;
CREATE VIEW country_quartile AS (
    WITH country_2016_forestation_percent AS (
        SELECT
            country_name,
            region,
            forest_area_sqkm,
            total_area_sqkm,
            ROUND(100 * forest_area_sqkm / total_area_sqkm, 3) AS forestation_percent
        FROM
            deforestation_tab
        WHERE
            year = 2016
            AND COALESCE(total_area_sqkm, 0) > 0 -- excluding countries having empty land_area_sqkm
            AND COALESCE(forest_area_sqkm, 0) > 0 -- excluding countries having empty forest_area_sqkm
        GROUP BY 1,2,3,4
        ORDER BY 5 DESC
    )

    SELECT
        country_name,
        region,
        forestation_percent,
        CASE
            WHEN forestation_percent BETWEEN 0 AND 25 THEN '1st Quartile (0-25%)'
            WHEN forestation_percent BETWEEN 25 AND 50 THEN '2nd Quartile (25-50%)'
            WHEN forestation_percent BETWEEN 50 AND 75 THEN '3rd Quartile (50-75%)'
            WHEN forestation_percent BETWEEN 75 AND 100 THEN '4th Quartile (75-100%)'
        END AS quartile
    FROM
        country_2016_forestation_percent
    ORDER BY 1
);


SELECT
    quartile,
    count(country_name)
FROM 
    country_quartile
WHERE
    country_name != 'World'
GROUP BY 1
ORDER BY 1

The largest number of countries in 2016 were found in the `1st quartile`.

There were `9` countries in the top quartile in 2016. These are countries with a very high percentage of their land area designated as forest. The following is a list of countries and their respective forest land, denoted as a percentage.


In [ ]:
%%sql

SELECT
    country_name,
    region,
    forestation_percent
FROM 
    country_quartile
WHERE
    quartile = '4th Quartile (75-100%)'
ORDER BY 3 DESC

## 4. RECOMMENDATIONS

### Group 1. Countries with SUCCESS STORIES: Find out insights and learn from them!

These countries are the role model! It would bring huge environmental benefits to study what has changed in this country over this time to drive higher forestation, and proliferate into other country groups below.

### Group 2. Countries with LARGEST CONCERNS: Prioritize improvement!

These countries need help and attention. It would be necessary to find out what has caused the issues and how to cure the situation. Learn from Group 1!

### Group 3. Countries in top forestation quartile: Preserve carefully.

These countries must be protected with high carefulness. It is necessary to prevent them from falling into Group 2.

### Group 4: Countries in bottom forestation quartile: Maintain and seek opportunity.

There is nothing much to be done if these countries are located where climate does not really support, except maintaining normal level of forestation. However if the climate supports, learn from Group 1.